# Product Quantization for Similarity Search

## How to compress and fit a humongous set of vectors in memory for similarity search with asymmetric distance computation (ADC)

### [Click here to read and learn how Product Quantization works (with detailed explanation and illustrations)](https://peggy1502.medium.com/2f1f67c5fddd)


In [1]:
import numpy as np
from scipy.cluster.vq import kmeans2, vq
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

# PQ functions for train, encode, search

- M = number of segments
- k = number of centroids per segment
- s = dimension, or length of a segment

In [2]:
def PQ_train(vectors, M, k):
    s = int(vectors.shape[1] / M)                      # dimension (or length) of a segment.
    codebook = np.empty((M, k, s), np.float32)         
        
    for m in range(M):
        sub_vectors = vectors[:, m*s:(m+1)*s]          # sub-vectors for segment m.
        codebook[m], label = kmeans2(sub_vectors, k)   # run k-means clustering for each segment.
        
    return codebook        

In [3]:
def PQ_encode(vectors, codebook):
    M, k, s = codebook.shape
    PQ_code = np.empty((vectors.shape[0], M), np.uint8)
    
    for m in range(M):
        sub_vectors = vectors[:, m*s:(m+1)*s]           # sub-vectors for segment m.
        centroid_ids, _ = vq(sub_vectors, codebook[m])  # vq returns the nearest centroid Ids.
        PQ_code[:, m] = centroid_ids                    # assign centroid Ids to PQ_code.
        
    return PQ_code

In [4]:
def PQ_search(query_vector, codebook, PQ_code):
    M, k, s = codebook.shape
    #=====================================================================
    # Build the distance table.
    #=====================================================================
    
    distance_table = np.empty((M, k), np.float32)    # Shape is (M, k)    
        
    for m in range(M):
        query_segment = query_vector[m*s:(m+1)*s]    # query vector for segment m.
        distance_table[m] = cdist([query_segment], codebook[m], "sqeuclidean")[0]
        
    #=====================================================================
    # Look up the partial distances from the distance table.
    #=====================================================================
    
    N, M = PQ_code.shape
    distance_table = distance_table.T               # Transpose the distance table to shape (k, M)
    distances = np.zeros((N, )).astype(np.float32)

    for n in range(N):                              # For each PQ Code, lookup the partial distances.
        for m in range(M):
            distances[n] += distance_table[PQ_code[n][m]][m] # Sum the partial distances from all the segments.
            
    return distance_table, distances    

In [5]:
# def PQ_search(query_vector, codebook, PQ_code):
#     M, k, s = codebook.shape
#     distance_table = np.empty((M, k), np.float32)              # Shape is (M, k)   
    
#     for m in range(M):
#         query_segment = query_vector[m*s:(m+1)*s]               # query vector for segment m.
        
#         distance_table[m] = cdist([query_segment], codebook[m], "sqeuclidean")[0]
# #       distance_table[m] = np.linalg.norm(codebook[m] - query_segment, axis=1) ** 2
        
#     distances = np.sum(distance_table[range(M), PQ_code], axis=1)
    
#     return distances    


## Examples of database vectors (of length 128) that will be divided and split into 8 segments, with 256 centroids per segment.

#### This test case shows example of results returned by using residual vectors compared to the vanilla PQ.

## For more information on why residual vectors are used in IVFPQ, read [Similarity Search with IVFPQ](https://towardsdatascience.com/similarity-search-with-ivfpq-9c6348fd4db3).

In [6]:
from sklearn.cluster import KMeans

M = 8
k = 256
vector_dim = 128          # Dimension (length) of a vector
top_n = 10

matches_PQ = 0
matches_res = 0
loop = 0

for total_vectors in [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]:
    loop += 1
    
    # === (A) Using vanilla PQ ===
    np.random.seed(2022)
    vectors = np.random.random((total_vectors, vector_dim)).astype(np.float32)   # Database vectors
    q = np.random.random((vector_dim, )).astype(np.float32)                      # Query vector

    codebook = PQ_train(vectors, M, k)
    PQ_code = PQ_encode(vectors, codebook)
    distance_table, distances = PQ_search(q, codebook, PQ_code)        
    PQ_top_n = distances.argsort()[:top_n]
     
    # === (B) Using Residual Vectors ===
    model = KMeans(n_clusters=1, init="k-means++", max_iter=300, n_init=10, random_state=0)  # 1 partition only (or virtually no partition).
    clusters = model.fit_predict(vectors)    
    partition_centroids = model.cluster_centers_
    residual_vectors = vectors - partition_centroids                            # Residual vectors
    qres = q - partition_centroids[0]                                           # Query residual

    codebook = PQ_train(residual_vectors, M, k)
    PQ_code = PQ_encode(residual_vectors, codebook)
    distance_table, distances = PQ_search(qres, codebook, PQ_code)

    res_top_n = distances.argsort()[:top_n]
    res_top_n
    
    # === (C) Actual Results ===
    N, _ = vectors.shape
    distances = np.zeros((N, )).astype(np.float32)

    for i, v in enumerate(vectors):
        distances[i] = cdist([v], [q], "sqeuclidean")[0]

    actual_top_n = distances.argsort()[:top_n]
    
    # === Find matches ===
    print("Total vectors:", total_vectors)
    print(f"List containing indices of top {top_n} nearest neighbors:")
    print("Actual Result    :", actual_top_n)
    
    matches = [x for x in PQ_top_n if x in actual_top_n]    
    print("Using vanilla PQ :", PQ_top_n, "Indices matching actual result =>", matches)
    matches_PQ = matches_PQ + len(matches)
    
    matches = [i for i in res_top_n if i in actual_top_n]       
    print("Using residuals  :", res_top_n, "Indices matching actual result =>", matches)
    matches_res = matches_res + len(matches)
        
    print("============================================")   

print("Average matches using vanilla PQ       :", round(matches_PQ / loop, 4))
print("Average matches using residual vectors :", round(matches_res / loop, 4))

/opt/conda/lib/python3.7/site-packages/scipy/cluster/vq.py:607: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "
/opt/conda/lib/python3.7/site-packages/scipy/cluster/vq.py:607: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


Total vectors: 1000
List containing indices of top 10 nearest neighbors:
Actual Result    : [ 75 302 940 139 218 285 971 498 902 752]
Using vanilla PQ : [380 576 946 569 106 626 389 906 860 350] Indices matching actual result => []
Using residuals  : [559 392 218 330  75 139 609 498 652 999] Indices matching actual result => [218, 75, 139, 498]


/opt/conda/lib/python3.7/site-packages/scipy/cluster/vq.py:607: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


Total vectors: 2000
List containing indices of top 10 nearest neighbors:
Actual Result    : [1735  188  584 1326 1316 1048 1046  242 1198 1754]
Using vanilla PQ : [ 446 1709  550  584  892 1735 1717  987  422 1287] Indices matching actual result => [584, 1735]
Using residuals  : [1198   50  188  242  286  747 1709  420  970  383] Indices matching actual result => [1198, 188, 242]


/opt/conda/lib/python3.7/site-packages/scipy/cluster/vq.py:607: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


...

Total vectors: 8000
List containing indices of top 10 nearest neighbors:
Actual Result    : [7772  813  150 3388 3687 5169 2285 5940 3325 5606]
Using vanilla PQ : [2982  202 2555 5598 4589 2149 1693 7448 3055 6784] Indices matching actual result => []
Using residuals  : [4827 2285  934 3079 3325 4580  150 7131 3851 2206] Indices matching actual result => [2285, 3325, 150]


/opt/conda/lib/python3.7/site-packages/scipy/cluster/vq.py:607: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


Total vectors: 9000
List containing indices of top 10 nearest neighbors:
Actual Result    : [8213 4966 5718 3227  932 6198 3305 1918 1784 5551]
Using vanilla PQ : [3640 3227 8492 8112 1503 1377 5592 1802 3357 8576] Indices matching actual result => [3227]
Using residuals  : [4935 5551 6508  400 6546 6421 3563 8131 4767 3602] Indices matching actual result => [5551]


/opt/conda/lib/python3.7/site-packages/scipy/cluster/vq.py:607: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


Total vectors: 10000
List containing indices of top 10 nearest neighbors:
Actual Result    : [8612 5135 8656 6441  735 7711 2644 3226 4479 2551]
Using vanilla PQ : [8002 9874 6088 1156 4816 5105 5135 7722 9773 1192] Indices matching actual result => [5135]
Using residuals  : [ 166 4964 7507 3560 6231 7351 8957 2644 6389 4555] Indices matching actual result => [2644]
Average matches using vanilla PQ       : 1.5
Average matches using residual vectors : 1.9
